In [33]:
import backtrader as bt
import pandas as pd
import numpy as np

In [34]:


class SimpleMovingAverageStrategy(bt.Strategy):
    params = (
        ("short_period", 5),
        ("long_period", 20),
        ("risk_factor", 0.01),
    )

    def __init__(self):
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_period)
        self.crossover = bt.indicators.CrossOver(self.short_ma, self.long_ma)
        self.bar_executed = None

    def next(self):
        if self.crossover > 0 and not self.position:  # Golden cross - short MA crosses above long MA
            amount_to_invest = (self.broker.get_cash() * self.params.risk_factor)
            self.size = amount_to_invest / self.data.close
            self.buy(size=self.size)
            self.bar_executed = len(self)

        if self.position and (self.crossover < 0 or len(self) - self.bar_executed >= 5):  # Death cross or position open for 5 days
            self.close()
            self.bar_executed = None

if __name__ == "__main__":
    # Load data
    df = pd.read_csv("USD_PLN Historical Data.csv")

    # Ensure the 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Drop the "Vol." column if it exists
    if "Vol." in df.columns:
        df.drop("Vol.", axis=1, inplace=True)

    # Remove NaN and infinite values
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    df.set_index('Date', inplace=True)
    df = df.iloc[::-1]  # Reverse the dataframe to have oldest data first
    data = bt.feeds.PandasData(dataname=df)

    # Create a Backtrader Cerebro engine
    cerebro = bt.Cerebro()

    # Add data and strategy to the engine
    cerebro.adddata(data)
    cerebro.addstrategy(SimpleMovingAverageStrategy)

    # Set broker parameters
    cerebro.broker.set_cash(10000)  # Starting cash
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

    # Print starting conditions
    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

    # Run the strategy
    cerebro.run()

    # Print final conditions
    print("Ending Portfolio Value: %.2f" % cerebro.broker.getvalue())

    # Plot the results
    cerebro.plot()



Starting Portfolio Value: 10000.00
Ending Portfolio Value: nan


ValueError: Axis limits cannot be NaN or Inf